In [1]:
import gitlab
from git import Repo
import os
import pandas as pd
import time

## Accessing the student solutions

Import the list of repo URLS for each milestone:

In [3]:
milestone_lists = [pd.read_csv("student_repos_list/st2m0.csv"),
                   pd.read_csv("student_repos_list/st2m1.csv"),
                   pd.read_csv("student_repos_list/st2m2.csv"),
                   pd.read_csv("student_repos_list/st2m3.csv"),
                   pd.read_csv("student_repos_list/st2m4.csv")]
milestone_names = ["m0", "m1", "m2", "m3", "m4"]

Clone every solution repo:

In [4]:
token = os.environ.get("GITLAB_ACCESS_TOKEN")
gl = gitlab.Gitlab('https://git.st.archi-lab.io/', private_token=token)

for milestone, milestone_name in zip(milestone_lists, milestone_names):
    for repo, student_name in zip(milestone["Code Repo"], milestone["Student"]):
        repo_url = repo + ".git"
        Repo.clone_from(repo_url, f"raw_solutions/student_solutions/{milestone_name}_{student_name.strip()}")
        time.sleep(0.06)

Check the repos were cloned correctly:

In [5]:
# generate all of the unique repos that were cloned
# list the milestone and student names from the directory
cloned_repos = pd.Series(os.listdir("raw_solutions/student_solutions")).str.split("_", expand=True)
cloned_repos = cloned_repos.rename(columns={0:"Milestone", 1:"Student"}).sort_values(["Milestone", "Student"]).reset_index(drop=True)

In [6]:
# generate all of the unique repos from the list of URLS
# add the milestone name to student name \ for every milestone
URL_repos = pd.concat([pd.concat([pd.Series([milestone_name]*len(milestone["Student"])), milestone["Student"].str.strip()], axis=1)\
                       for milestone_name, milestone in zip(milestone_names, milestone_lists)])
URL_repos = URL_repos.rename(columns={0:"Milestone"}).sort_values(["Milestone", "Student"]).reset_index(drop=True)

In [7]:
cloned_repos.equals(URL_repos)

True